In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualizations
import os
import glob
import time
import sys

In [101]:
with open("/home/dunbar/Downloads/NC_000019.10[1..6900000].fa") as fd:
    file = fd.read()
    fd.close()

len(file)
file = file.split("Assembly")[1]
file = file.split("\n")
temp = ''
[temp.join(sub) for sub in file]
file = temp
step = 16
words = []
for i in range(0,len(file)-step,step):
    j = i+step
    words.append(file[i:j])

Count Min Sketch Ideas:

1) Show Frequency Count efficiency (Naive -> CMS implementation)
    - Basic Dictionary Count
    - Binning applications (When we can say "close enough" and clump instances togeather)

2) Show application to specific problems (Anomaly Detection

In [51]:
# naively count frequencies

class dictionary():
    
    def __init__(self):
        self.dictionary = {}
        self.nbytes = sys.getsizeof(self)
    
    def getsize(self):
        print("Dictionary is Size: {} Bytes\n".format(self.nbytes))
        
    def add(self,token):
        if token in self.dictionary:
            self.dictionary[token] += 1
        else:
            self.dictionary[token] = 1
        self.nbytes = sys.getsizeof(self.dictionary)
        
    def timed_update(self,tokenlist):
        startsize = self.nbytes
        start = time.time()
        for token in tokenlist:
            self.add(token)
        end = time.time() - start
        dsize = self.nbytes - startsize
        print("Time Elapsed: {} Seconds \n".format(end))
        print("Change In Memory: {} Bytes\n".format(dsize))
    
    def estimate(self,token):
        try:
            return self.dictionary[token]
        except:
            print("Error: Token Not Found \n")

In [105]:

class CountMinSketch:
    
    def __init__(self,vec_length=2**4):
        self.N = vec_length

        self.PRIMES = [121021, 121151, 150151, 151051, 151121, 180181, 180811, 181081, 2976221, 
          3021377, 6972593, 13466917, 20996011, 24036583, 25964951, 30402457,33923743,33492811,77381235]
        self.table = np.zeros((len(self.PRIMES),self.N))
        self.hashes = [self._genhash(prime) for prime in self.PRIMES]
        self.nbytes = sys.getsizeof(self.table) + sys.getsizeof(self.hashes)
        
    def _genhash(self,prime):
        p = prime
        def hash_fn(val):
            key = ['A','C','G','T']
            val = np.array([key.index(val[i]) for i in range(len(val))])
            index = (val*p)
            return index%len(self.PRIMES)
        return hash_fn

    def getsize(self):
        print("Sketch is Size: {} Bytes\n".format(self.nbytes))
        
    def add(self, val):
      
        for ix in range(0, self.N):
            h = self.hashes[ix](val)
            self.table[h][ix] += 1
        self.nbytes = sys.getsizeof(self.table) + sys.getsizeof(self.hashes)
            
    def timed_update(self,valuelist):
        startsize = self.nbytes
  
    
        start = time.time()
        for value in valuelist:
            self.add(value)
        end = time.time() - start
        dsize = sys.getsizeof(self.table) + sys.getsizeof(self.hashes) - startsize
        print("Time Elapsed: {} Seconds \n".format(end))
        print("Change In Memory: {} Bytes\n".format(dsize))
                              
                              
       
    def count(self,val):
        # Helper Function
        vals = []
        for ix in range(0, N):
            h = self.hashes[ix](val)
            vals.append(self.table[h][ix])
        return vals
            
    def estimate(self, value):
   
        results = []
        for ix in range(0, N):
            h = self.hashes[ix](value)
            c = self.table[h][ix]
            results.append(c)
        return min(results)

In [49]:
class changeDetection():
    def __init__(self,sketchset,w):
        self.sketchset = sketchset
        self.w = w
        self.MA = None
        
    def update(self,sketch):
        self.sketchset.append(sketch)
        if len(self.sketchet >self.w):
            self.sketchset.pop(0)
        self.calcmovingAv()
            
    def calcmovingAv(self):
        tableav = 0
        for i,sketch in enumerate(self.sketchset):
            tableav += sketch.table
        tableav /= len(self.sketchset)
        self.MA = tableav
        
    def indmovingAv(self,val):
        # Helper Function
        mavals = []
        for ix in range(0, N):
            h = self.sketchset[-1].hashes[ix](val)
            mavals.append(self.MA[h][ix])
        return mavals
    
    def bucketDist(self,value,idx):
        # Helper Function
        sketch = self.sketchset[idx]
        K = np.sum(sketch.table > 0)
        counts = sketch.count(vals)
        vals = (counts - self.indmovingAv(value))
        vals = [value/(1 - 1/K) for value in vals]
        v_a  = np.median(vals)
        return counts,v_a
    
    def isAttack(self,value,beta):
        # given a set of values, compute their respective counts and determine the
        # "Change" in the count frequency relative to their respective bins to dermine 
        # an anomoly
        
        # If the ID post count exceeds the criteria from the previous sketch
        # then it qualifies as an attack
        counts,variance = self.bucketDist(value,-1)
        mean = self.MA
        crit = mean + beta*variance
        for count in counts:
            if count > crit:
                print("Found Anomoly: {},{}".format(count,crit))
                return True
          

In [52]:
genomedict = dictionary()
genomedict.timed_update(words)

Time Elapsed: 0.23912620544433594 Seconds 

Change In Memory: 20971560 Bytes



In [106]:
genomesketch = CountMinSketch()
genomesketch.timed_update(words)

Time Elapsed: 4.76837158203125e-07 Seconds 

Change In Memory: 0 Bytes

